<a href="https://colab.research.google.com/github/Pravin98651/LLM/blob/main/Training_Hack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
!pip install -q pdfplumber python-docx faiss-cpu sentence-transformers


In [26]:
from google.colab import files
uploaded = files.upload()


Saving ICIHLIP22012V012223.pdf to ICIHLIP22012V012223 (1).pdf
Saving HDFHLIP23024V072223.pdf to HDFHLIP23024V072223 (1).pdf
Saving EDLHLGA23009V012223.pdf to EDLHLGA23009V012223 (1).pdf
Saving BAJHLIP23020V012223.pdf to BAJHLIP23020V012223 (1).pdf
Saving CHOTGDP23004V012223.pdf to CHOTGDP23004V012223 (1).pdf


In [27]:
import pdfplumber
import docx
from typing import List

def extract_text_from_pdf(pdf_path: str) -> str:
    text = ''
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + '\n'
    return text

def extract_text_from_docx(docx_path: str) -> str:
    doc = docx.Document(docx_path)
    return '\n'.join([p.text for p in doc.paragraphs])

def chunk_text(text: str, chunk_size: int = 500, overlap: int = 50) -> List[str]:
    words = text.split()
    return [' '.join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size - overlap)]


In [28]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

embed_model = SentenceTransformer('all-MiniLM-L6-v2')

def embed_and_index(chunks: List[str]):
    embeddings = embed_model.encode(chunks, show_progress_bar=True)
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(np.array(embeddings))
    return index, embeddings


In [29]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
import os
import pdfplumber
import docx
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

def extract_text_from_pdf(pdf_path: str) -> str:
    text = ''
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + '\n'
    return text

def extract_text_from_docx(docx_path: str) -> str:
    doc = docx.Document(docx_path)
    return '\n'.join([p.text for p in doc.paragraphs])

def chunk_text(text: str, chunk_size: int = 500, overlap: int = 50):
    words = text.split()
    return [' '.join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size - overlap)]


In [31]:
folder_path = '/content/drive/MyDrive/Training Docs'  # change if your folder is nested deeper
all_chunks = []

for fname in os.listdir(folder_path):
    file_path = os.path.join(folder_path, fname)
    if fname.endswith('.pdf'):
        print(f"📄 Reading PDF: {fname}")
        text = extract_text_from_pdf(file_path)
    elif fname.endswith('.docx'):
        print(f"📝 Reading DOCX: {fname}")
        text = extract_text_from_docx(file_path)
    else:
        print(f"⚠️ Skipping unsupported file: {fname}")
        continue

    chunks = chunk_text(text)
    print(f"✅ Extracted {len(chunks)} chunks from {fname}")
    all_chunks.extend(chunks)


📄 Reading PDF: CHOTGDP23004V012223.pdf
✅ Extracted 117 chunks from CHOTGDP23004V012223.pdf
📄 Reading PDF: BAJHLIP23020V012223.pdf
✅ Extracted 65 chunks from BAJHLIP23020V012223.pdf
📄 Reading PDF: EDLHLGA23009V012223.pdf
✅ Extracted 3 chunks from EDLHLGA23009V012223.pdf
📄 Reading PDF: HDFHLIP23024V072223.pdf
✅ Extracted 41 chunks from HDFHLIP23024V072223.pdf
📄 Reading PDF: ICIHLIP22012V012223.pdf
✅ Extracted 48 chunks from ICIHLIP22012V012223.pdf


In [32]:
embed_model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = embed_model.encode(all_chunks, show_progress_bar=True)

dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(np.array(embeddings))

print(f"\n✅ FAISS Index created with {len(all_chunks)} total chunks.")


Batches:   0%|          | 0/9 [00:00<?, ?it/s]


✅ FAISS Index created with 274 total chunks.


In [33]:
def semantic_search(query: str, k: int = 5):
    query_vec = embed_model.encode([query])
    distances, indices = index.search(np.array(query_vec), k)
    return [all_chunks[i] for i in indices[0]]

query = "Does this policy cover knee surgery?"
results = semantic_search(query)

for i, res in enumerate(results):
    print(f"\n🔍 Result {i+1}:\n{res}")




🔍 Result 1:
under the Policy. The payment under this cover specific exclusion. is subject to admissibility of Your In-patient treatment Claim under the Policy. e. If the Insured Person is continuously covered without any break as defined d. Exclusions under the applicable norms on portability We will not be liable for any Voluntary Deductible stipulated by IRDAI, then waiting period amount, if applicable and as specifically defined in the for the same would be reduced to the Policy Schedule under the Policy. extent of prior coverage. We will not be liable to make any payment under this Policy in connection with or in respect of any expenses whatsoever incurred in connection with or in respect of: UIN : ICIHLIP22012V012223 17 CIN: L67200MH2000PLC129408 Golden Shield f. List of specific Illness and Surgical Procedures as mention below: Sr. Organ /Organ Illness/ diagnosis Surgeries/ Procedures (irrespective of any No. System (irrespective of treatments medical or surgical) illness or dia

In [34]:
import pickle

# Paths to save
save_dir = "/content/drive/MyDrive/TrainingModel"  # You can choose another subfolder
os.makedirs(save_dir, exist_ok=True)

# Save FAISS index
faiss.write_index(index, os.path.join(save_dir, "index.faiss"))

# Save chunks
with open(os.path.join(save_dir, "chunks.pkl"), "wb") as f:
    pickle.dump(all_chunks, f)

print(f"✅ Model saved to Google Drive in: {save_dir}")


✅ Model saved to Google Drive in: /content/drive/MyDrive/TrainingModel


In [35]:
# Mount Drive again if in new session
from google.colab import drive
drive.mount('/content/drive')

import faiss
import pickle

load_dir = "/content/drive/MyDrive/TrainingModel"

# Load FAISS index
index = faiss.read_index(os.path.join(load_dir, "index.faiss"))

# Load chunks
with open(os.path.join(load_dir, "chunks.pkl"), "rb") as f:
    all_chunks = pickle.load(f)

print("✅ Model and data successfully loaded.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Model and data successfully loaded.


In [36]:
import os
import numpy as np
import faiss
import pickle

save_dir = "/content/drive/MyDrive/TrainingModel"
os.makedirs(save_dir, exist_ok=True)

# Save FAISS index
faiss.write_index(index, os.path.join(save_dir, "index.faiss"))

# Save chunks
with open(os.path.join(save_dir, "chunks.pkl"), "wb") as f:
    pickle.dump(all_chunks, f)

# Save raw embeddings
np.save(os.path.join(save_dir, "embeddings.npy"), embeddings)

print(f"✅ FAISS index, chunks, and embeddings saved to {save_dir}")



✅ FAISS index, chunks, and embeddings saved to /content/drive/MyDrive/TrainingModel


In [37]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import faiss
import pickle

load_dir = "/content/drive/MyDrive/TrainingModel"

# Load FAISS index
index = faiss.read_index(os.path.join(load_dir, "index.faiss"))

# Load chunks
with open(os.path.join(load_dir, "chunks.pkl"), "rb") as f:
    all_chunks = pickle.load(f)

# Load raw embeddings
embeddings = np.load(os.path.join(load_dir, "embeddings.npy"))

print(f"✅ Loaded {len(all_chunks)} chunks and embeddings with shape: {embeddings.shape}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Loaded 274 chunks and embeddings with shape: (274, 384)


In [38]:
def precision_at_k(query, ground_truth_answer, k=5):
    query_embedding = model.encode([query])
    D, I = index.search(query_embedding, k)

    retrieved_chunks = [all_chunks[i] for i in I[0]]
    hits = sum(1 for chunk in retrieved_chunks if ground_truth_answer.lower() in chunk.lower())
    return hits / k


In [39]:
import numpy as np
import pickle
import faiss
from sentence_transformers import SentenceTransformer

# STEP 1: Load saved artifacts from Google Drive
embeddings_path = "/content/drive/MyDrive/TrainingModel/embeddings.npy"
chunks_path = "/content/drive/MyDrive/TrainingModel/chunks.pkl"
index_path = "/content/drive/MyDrive/TrainingModel/index.faiss"

# Load chunks
with open(chunks_path, 'rb') as f:
    all_chunks = pickle.load(f)

# Load embeddings
embeddings = np.load(embeddings_path)

# Load FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
faiss.read_index(index_path)  # Or use: index = faiss.read_index(index_path)

# Load embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# STEP 2: Define precision@k evaluation
def precision_at_k(query, ground_truth_answer, k=3):
    query_embedding = model.encode([query])
    D, I = index.search(query_embedding, k)

    retrieved_chunks = [all_chunks[i] for i in I[0]]
    print("🔍 Top Retrieved Chunks:")
    for chunk in retrieved_chunks:
        print("-", chunk[:300], "...\n")  # Limit output length for readability

    hits = sum(1 for chunk in retrieved_chunks if ground_truth_answer.lower() in chunk.lower())
    precision = hits / k
    return precision

# STEP 3: Sample Evaluation
query = "Knee"


ground_truth = "30-day grace period for premium payment"

score = precision_at_k(query, ground_truth, k=3)
print(f"\n🎯 Precision@3: {score:.2f}")


🔍 Top Retrieved Chunks:
- Nagaland and Email: bimalokpal.mumbai@cioins.co.in (ASSAM). Tripura. Tel.: 0361 - 2632204 / 2602205 Email: bimalokpal.guwahati@cioins.co.in HYDERABAD Andhra Pradesh, Office of the Insurance Ombudsman, Telangana, Yanam 6-2-46, 1st floor, “Moin Court”, Lane Opp. and part of Union Saleem Function Palac ...

- Nagaland and Email: bimalokpal.mumbai@cioins.co.in (ASSAM). Tripura. Tel.: 0361 - 2632204 / 2602205 Email: bimalokpal.guwahati@cioins.co.in HYDERABAD Andhra Pradesh, Office of the Insurance Ombudsman, Telangana, Yanam 6-2-46, 1st floor, “Moin Court”, Lane Opp. and part of Union Saleem Function Palac ...

- Nagaland and Email: bimalokpal.mumbai@cioins.co.in (ASSAM). Tripura. Tel.: 0361 - 2632204 / 2602205 Email: bimalokpal.guwahati@cioins.co.in HYDERABAD Andhra Pradesh, Office of the Insurance Ombudsman, Telangana, Yanam 6-2-46, 1st floor, “Moin Court”, Lane Opp. and part of Union Saleem Function Palac ...


🎯 Precision@3: 0.00


In [41]:
for i, chunk in enumerate(all_chunks):
    if "grace period" in chunk.lower():
        print(f"\n🔎 Found keyword in chunk {i}: {chunk[:300]}...\n")



🔎 Found keyword in chunk 3: care in scope, duration, or intensity; iii. must have been prescribed by a Medical Practitioner; must conform to the professional standards widely accepted in international medical practice or by the medical community in India. Network Provider/ Hospital means Hospitals or health care providers enli...


🔎 Found keyword in chunk 26: Carrier, Daily allowance in case of Hospitalization Sections. 21. Notice of charge: The Company shall not be bound to take notice or be affected by any notice of any trust, charge, lien, assignment or other dealing with or relating to this Policy, but the payment by the Company to the Insured /Insur...


🔎 Found keyword in chunk 119: or a medical set -up with a Hospital and which has been registered with the local authorities, wherever applicable, and is under supervision of a registered and qualified Medical Practitioner AND must comply with all minimum criterion as under:- i. has qualified nursing staff under its employment; i

In [46]:
eval_set = [
    {"query": "What is the grace period for premium payment?", "answer": "30-day grace period"},
    {"query": "What are pre-existing conditions?", "answer": "Conditions that existed before the start of the policy"},
    {"query": "What is the waiting period for maternity benefits?", "answer": "36 months"},
    ...
]


In [44]:
def evaluate_precision(eval_set, k=3):
    scores = []
    for item in eval_set:
        p = precision_at_k(item["query"], item["answer"], k)
        scores.append(p)
        print(f"Query: {item['query']}\nAnswer: {item['answer']}\nPrecision@{k}: {p:.2f}\n")
    avg = sum(scores) / len(scores)
    print(f"\n📊 Average Precision@{k}: {avg:.2f}")


In [48]:
evaluate_precision(eval_set, k=3)


🔍 Top Retrieved Chunks:
- Nagaland and Email: bimalokpal.mumbai@cioins.co.in (ASSAM). Tripura. Tel.: 0361 - 2632204 / 2602205 Email: bimalokpal.guwahati@cioins.co.in HYDERABAD Andhra Pradesh, Office of the Insurance Ombudsman, Telangana, Yanam 6-2-46, 1st floor, “Moin Court”, Lane Opp. and part of Union Saleem Function Palac ...

- Nagaland and Email: bimalokpal.mumbai@cioins.co.in (ASSAM). Tripura. Tel.: 0361 - 2632204 / 2602205 Email: bimalokpal.guwahati@cioins.co.in HYDERABAD Andhra Pradesh, Office of the Insurance Ombudsman, Telangana, Yanam 6-2-46, 1st floor, “Moin Court”, Lane Opp. and part of Union Saleem Function Palac ...

- Nagaland and Email: bimalokpal.mumbai@cioins.co.in (ASSAM). Tripura. Tel.: 0361 - 2632204 / 2602205 Email: bimalokpal.guwahati@cioins.co.in HYDERABAD Andhra Pradesh, Office of the Insurance Ombudsman, Telangana, Yanam 6-2-46, 1st floor, “Moin Court”, Lane Opp. and part of Union Saleem Function Palac ...

Query: What is the grace period for premium payment?

TypeError: 'ellipsis' object is not subscriptable

In [49]:
for i, chunk in enumerate(chunks[:5]):
    print(f"🔹 Chunk {i}: {chunk[:300]}...\n")


🔹 Chunk 0: For Buy/ Renew/ Service/ Claim related queries Log on to www.icicilombard.com or call 1800 2666 Golden Shield Policy Wording b. Preamble theatre where surgical procedures are to be carried out; This Policy has been issued on the basis of the Disclosure to information Norm, including the information ...

🔹 Chunk 1: treatment undergone by the insured in accordance medical/surgical/para-surgical treatment procedures with the policy terms and conditions, are directly made and interventions are carried out by AYUSH Medical to the network provider by the insurer to the extent pre- practitioner(s) comprising of any ...

🔹 Chunk 2: has qualified medical practitioner/s in charge 2010 or under enactments specified under the c. has fully equipped operation theatre of its own Schedule of Section 56(1) of the said Act Or comply where surgical procedures are carried out; with all minimum criteria as under: d. maintains daily records...

🔹 Chunk 3: illness/disease/injury which in the norma

In [50]:
eval_set = [
    {
        "query": "What is the grace period for premium payment?",
        "answer": "30 days",
    },
    {
        "query": "What is the waiting period for maternity benefits?",
        "answer": "36 months",
    },
    {
        "query": "What are pre-existing conditions?",
        "answer": "Any disease or illness existing prior to the start of the policy",
    },
    {
        "query": "What is portability in health insurance?",
        "answer": "Right to transfer policy with continuity benefits",
    },
]


In [51]:
evaluate_precision(eval_set, k=3)


🔍 Top Retrieved Chunks:
- Nagaland and Email: bimalokpal.mumbai@cioins.co.in (ASSAM). Tripura. Tel.: 0361 - 2632204 / 2602205 Email: bimalokpal.guwahati@cioins.co.in HYDERABAD Andhra Pradesh, Office of the Insurance Ombudsman, Telangana, Yanam 6-2-46, 1st floor, “Moin Court”, Lane Opp. and part of Union Saleem Function Palac ...

- Nagaland and Email: bimalokpal.mumbai@cioins.co.in (ASSAM). Tripura. Tel.: 0361 - 2632204 / 2602205 Email: bimalokpal.guwahati@cioins.co.in HYDERABAD Andhra Pradesh, Office of the Insurance Ombudsman, Telangana, Yanam 6-2-46, 1st floor, “Moin Court”, Lane Opp. and part of Union Saleem Function Palac ...

- Nagaland and Email: bimalokpal.mumbai@cioins.co.in (ASSAM). Tripura. Tel.: 0361 - 2632204 / 2602205 Email: bimalokpal.guwahati@cioins.co.in HYDERABAD Andhra Pradesh, Office of the Insurance Ombudsman, Telangana, Yanam 6-2-46, 1st floor, “Moin Court”, Lane Opp. and part of Union Saleem Function Palac ...

Query: What is the grace period for premium payment?

In [52]:
print(f"Number of chunks: {len(chunks)}")


Number of chunks: 48


In [54]:
from collections import Counter
# chunk_texts = [c['content'] for c in chunks]
duplicates = Counter(chunks).most_common()
print("Top repeated chunks:\n", duplicates[:5])

Top repeated chunks:
 [('For Buy/ Renew/ Service/ Claim related queries Log on to www.icicilombard.com or call 1800 2666 Golden Shield Policy Wording b. Preamble theatre where surgical procedures are to be carried out; This Policy has been issued on the basis of the Disclosure to information Norm, including the information provided by iv. Maintaining daily records of the patients and Proposer in respect of the Insured Persons in the Proposal making them accessible to the insurance Form, any application for insurance cover in respect of any company’s authorized representative Insured Person and any other information or details AYUSH Day Care Centre means and includes submitted in relation to the Proposal Form. This Policy is a Community Health Centre (CHC), Primary Health contract of insurance between You and Us which is subject Centre (PHC), Dispensary, Clinic, Polyclinic or any such to the receipt of premium in full and accepted by Us in health centre which is registered with the loca

In [55]:
print(f"Embeddings shape: {embeddings.shape}")


Embeddings shape: (274, 384)


In [57]:
from sklearn.metrics.pairwise import cosine_similarity
query_embedding = embed_model.encode(["What is the grace period for premium payment?"])[0]

# Compute cosine similarity
sims = cosine_similarity([query_embedding], embeddings)[0]
top_k = sims.argsort()[-5:][::-1]

for i in top_k:
    print(f"Similarity: {sims[i]:.4f} | Chunk: {all_chunks[i][:200]}...")

Similarity: 0.6405 | Chunk: at the time of renewal with all the accrued continuity benefits such as cumulative bonus, waiver of waiting period as per IRDAI guidelines, provided the Policy has been maintained without a break. 12....
Similarity: 0.6207 | Chunk: No interest will be charged If the installment premium is not paid on due date v. In case of installment premium due not received within the Grace Period, the Policy will get cancelled vi. In the even...
Similarity: 0.5480 | Chunk: commenced, such proportionate premium 13. Premium Payment in lnstalments (Wherever commensurate with the insurance coverage applicable) during such period; lf the insured person has opted for Payment ...
Similarity: 0.5241 | Chunk: the insured person, is in any Company, it shall initiate and complete such respect fraudulent, or if any false statement, or investigation at the earliest, in any case not declaration is made or used ...
Similarity: 0.5089 | Chunk: any kind/bamboo/plastic cloth/asphalt/canvas

In [60]:
# Define your query and expected relevant substrings
query = "What is the grace period for premium payment?"
expected_keywords = ["grace period", "premium due", "installment", "policy cancelled", "waiver"]

# Check how many of the top_k retrieved chunks contain expected keywords
relevant = 0
for i in top_k:
    content = all_chunks[i].lower()
    if any(keyword in content for keyword in expected_keywords):
        relevant += 1

precision_at_k = relevant / len(top_k)
print(f"Precision@{len(top_k)}: {precision_at_k:.2f}")


Precision@5: 0.60
